In [ ]:
import tensorflow as tf

In [ ]:
cd ..

In [ ]:
from data import helpers
from data import queryer
import copy
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
egdata = queryer.get_earthquake_eager()
print([k for k in egdata[0]])# for one earthquake here are the dictionarnum_py,y keys

In [ ]:
import pandas as pd
table = pd.read_csv('events.csv')
table
quake_lats = table.Latitude.as_matrix()
quake_lons = table.Longitude.as_matrix()
print(type(quake_lats))
print(quake_lats.shape)

#plot eq epicenter locations
plt.scatter(quake_lons[433:],quake_lats[433:],alpha=0.3,c='r',label='1911-2015')
plt.scatter(quake_lons[0:433],quake_lats[0:433],alpha=0.3,marker='^',label='2015-Present')
plt.legend()
#plot the grid boundaries
plen_x = 0.01 #-> angle that is covered by one pixel in x dir
plen_y = 0.01 #-> angle that is covered by one pixel in y dir
size_x = 0.8  #-> total covered angle in xdir (longitudes)
size_y = 0.45 #-> total covere angle in ydir (lattitudes)
topleft = np.asarray([6.4,53.50]) #-> initialize the top left part
botleft = topleft+[0,-size_y]
topright = topleft+[size_x,0]
botright = topleft+[size_x,-size_y]
print(topleft)
print(topright)

plt.plot([topleft[0],topright[0]],[topleft[1],topright[1]],'go--', linewidth=2, markersize=4)
plt.plot([topleft[0],botleft[0]],[topleft[1],botleft[1]],'go--', linewidth=2, markersize=4)
plt.plot([botleft[0],botright[0]],[botleft[1],botright[1]],'go--', linewidth=2, markersize=4)
plt.plot([botright[0],topright[0]],[botright[1],topright[1]],'go--', linewidth=2, markersize=4)
plt.scatter(egdata[0]['eventlon'],egdata[0]['eventlat'],marker='s',c='k')

plt.show()

#print(table.Magnitude[4000,:])
magnitude_array = table.Magnitude.as_matrix()
plt.plot(magnitude_array[:],marker='*')
plt.xlabel('Earthquake #')
plt.ylabel('Moment Mag')
plt.title(['max mag',np.amax(magnitude_array,0)])
print(magnitude_array.shape)
print(np.amax(magnitude_array,0))

plt.show()
magnitude_array[0:3]
np.argwhere(magnitude_array==0.5687999158)
print(quake_lats.shape)

In [ ]:
quake_trace = egdata[0]
quake_lat,quake_lon = quake_trace['eventlat'],quake_trace['eventlon']
print(quake_lat,quake_lon)
print(type(quake_lat),type(quake_lon))

#print(quake_trace)

In [ ]:
def discretiser(plen_x, plen_y, xy):
    """Args: 
    plen_x : pixel length(degree) in x dim
    plen_y : pixel length(degree) in y dim
    (x,y) real valued x,y coordinates where 0<x<size, 0<y<size
    Returns:
    (n,m) where n,m are integer valued pixel numbers e.g. (102,88)"""
    x,y = xy
    n = int(x/plen_x)
    m = int(y/plen_y)
    return n,m

def grid_translation(topleft,stationlat, stationlon):
    """Args: 
    topleft is a tuple of the (lat, lon) of the most NW part of our grid
    stationlat,stationlon are the coordinates of the location we want translated
    Returns:
    (x,y) real valued x,y coordinates where 0<x<size, 0<y<size"""
    lat_ref = topleft[0]
    lon_ref = topleft[1]
    dx = (stationlon-lon_ref)
    dy = (lat_ref-stationlat)
    return dx,dy

def get_pixel_lens(numx, numy, sizex, sizey):
    return sizex/numx, sizey/numy

topleft = np.asarray([53.50,6.4]) #-> initialize the top left part
size_x = 0.8  #-> total covered angle in xdir (longitudes)
size_y = 0.45 #-> total covere angle in ydir (lattitudes)

numx= 2
numy = 3


plen_x, plen_y = get_pixel_lens(numx, numy, size_x, size_y)
print('PIXEL LENS = ',plen_x,plen_y)
#plen_x = 0.01 #-> angle that is covered by one pixel in x dir
#plen_y = 0.01 #-> angle that is covered by one pixel in y dir

print(egdata[0]['eventid'])
print(egdata[0]['eventlat'])
print(egdata[0]['eventlon'])
print(egdata[0]['magnitude'])
print(egdata[0]['eventdepth'])
print(len(egdata[0]['data']))

quake_event = egdata[0]
quake_lat,quake_lon = [quake_event['eventlat'],quake_event['eventlon']]
print(quake_lat,quake_lon,type(quake_lat),type(quake_lon))
print('topleft=',topleft)
print('event_lat,event_lon = ',[quake_event['eventlat'],quake_event['eventlon']])
# find the grid indices of the event location (grid is m by n)
xy = grid_translation(topleft,quake_event['eventlat'],quake_event['eventlon'])
print(xy)
n,m = discretiser(plen_x,plen_y,xy)
print(n,m)
event_label_matrix = np.zeros((numy,numx))
event_label_matrix[m,n]=1.0
event_label_matrix_flatten = event_label_matrix.flatten();
print(event_label_matrix)
print(event_label_matrix_flatten)

print(event_label_matrix.shape)
print(event_label_matrix_flatten.shape)

print(90*180)
#print(quake_event)

In [ ]:
def _format_label(quake_event,topleft,numx,numy,size_x,size_y):
    """
    Args: label is like (53.223, 6.949, 3.0, 0.502994544, 'knmi2018etrn',...)
    (oneevent['eventlat'],oneevent['eventlon']
            ,oneevent['eventdepth'], oneevent['magnitude']
            ,oneevent['eventid'], topleft, sizex, sizey, numx,numy)
    Returns: onehot tile encoding - numpy array
    """
    xy = grid_translation(topleft,quake_event['eventlat'],quake_event['eventlon'])
    plen_x, plen_y = get_pixel_lens(numx, numy, size_x, size_y)
    n,m = discretiser(plen_x,plen_y,xy)
    event_label_matrix = np.zeros((numy,numx))
    event_label_matrix[m,n]=1.0
    event_label_matrix_flatten = event_label_matrix.flatten();

    return event_label_matrix_flatten

topleft = np.asarray([53.50,6.4]) #-> initialize the top left part
numx= 2
numy = 3
size_x = 0.8  #-> total covered angle in xdir (longitudes)
size_y = 0.45 #-> total covere angle in ydir (lattitudes)

flat_label_vec = _format_label(egdata[0],topleft,numx,numy,size_x,size_y)
print(flat_label_vec)